<a href="https://colab.research.google.com/github/Raph33AI/SonicSense-AI/blob/main/SonicSense.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:

import requests
from bs4 import BeautifulSoup

url = "https://charts.spotify.com/viral/global/weekly/latest"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

trending_titles = []
for track_row in soup.select("table tbody tr"):
    title_element = track_row.find("span", class_="chart-table-track-name")
    if title_element:
        title = title_element.text.strip()
        trending_titles.append(title)

print("The 100 most trending titles on Spotify this week:")
for i, title in enumerate(trending_titles):
    print(f"{i+1}. {title}")


The 100 most trending titles on Spotify this week:


In [29]:
import requests
import base64
import random

# Vos identifiants d'application
CLIENT_ID = "dffbe0f471154691b105df32348a2378"
CLIENT_SECRET = "cd114e5ad1654451a84a1d3b9e32c7ee"

# Fonction pour obtenir l'access token Spotify
def get_access_token():
    auth_string = f"{CLIENT_ID}:{CLIENT_SECRET}"
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    token_url = "https://accounts.spotify.com/api/token"
    token_data = {
        "grant_type": "client_credentials"
    }
    token_headers = {
        "Authorization": f"Basic {auth_base64}"
    }

    token_response = requests.post(token_url, data=token_data, headers=token_headers)

    if token_response.status_code == 200:
        return token_response.json()["access_token"]
    else:
        print(f"Erreur lors de la récupération du token: {token_response.status_code}")
        return None

# Obtenir l'access token
ACCESS_TOKEN = get_access_token()

if ACCESS_TOKEN:
    headers = {
        "Authorization": f"Bearer {ACCESS_TOKEN}"
    }

    genre_demandé = input("Entrez un genre musical : ")
    artiste_demandé = input("Entrez le nom d'un artiste : ")

    # Définir un offset aléatoire (par exemple, entre 0 et 100)
    offset_aleatoire = random.randint(0, 100)

    search_url = "https://api.spotify.com/v1/search"
    search_params = {
        "q": f"genre:{genre_demandé} artist:{artiste_demandé}", # Recherche par genre ET artiste
        "type": "track",
        "limit": 50,
        "offset": offset_aleatoire
    }
    search_response = requests.get(search_url, headers=headers, params=search_params)

    if search_response.status_code == 200:
        search_data = search_response.json()
        tracks = search_data["tracks"]["items"]

        print("Artiste - Album - Date - Tempo - Danceability - Energy - Valence - Lien")
        print("---------------------------------------------------------------------")

        for track in tracks:
            artists = ", ".join([artist["name"] for artist in track["artists"]])
            album = track["album"]["name"]
            release_date = track["album"]["release_date"]
            track_id = track['id']

            audio_features_url = f"https://api.spotify.com/v1/audio-features/{track_id}"
            audio_features_response = requests.get(audio_features_url, headers=headers)

            if audio_features_response.status_code == 200:
                audio_features = audio_features_response.json()
                tempo = audio_features.get("tempo", "N/A")
                danceability = audio_features.get("danceability", "N/A")
                energy = audio_features.get("energy", "N/A")
                valence = audio_features.get("valence", "N/A")

                print(
                    f"{artists} - {album} - {release_date} - "
                    f"{tempo:.2f} BPM - {danceability:.2f} - "
                    f"{energy:.2f} - {valence:.2f} - "
                    f"{track['external_urls']['spotify']}"
                )
            else:
                print(f"Erreur lors de la récupération des caractéristiques audio : {audio_features_response.status_code}")
    else:
        print(f"Erreur lors de la recherche de morceaux : {search_response.status_code}")

Entrez un genre musical : rap
Entrez le nom d'un artiste : Travis Scott
Artiste - Album - Date - Tempo - Danceability - Energy - Valence - Lien
---------------------------------------------------------------------
Young Thug, Travis Scott, Gunna - So Much Fun (Deluxe) - 2019-12-20 - 111.98 BPM - 0.84 - 0.63 - 0.55 - https://open.spotify.com/track/4ea9w8c4ROqiZpJVhfBA3m
Travis Scott, Rich Homie Quan, Young Thug - DAYS BEFORE RODEO - 2014-08-18 - 115.03 BPM - 0.80 - 0.86 - 0.24 - https://open.spotify.com/track/0uFz9q2SvuKyWB6Ur1YCfI
Travis Scott - Rodeo - 2015-09-04 - 131.05 BPM - 0.71 - 0.53 - 0.12 - https://open.spotify.com/track/1wHZx0LgzFHyeIZkUydNXq
Travis Scott - ASTROWORLD - 2018-08-03 - 150.01 BPM - 0.49 - 0.79 - 0.06 - https://open.spotify.com/track/7wBJfHzpfI3032CSD7CE2m
Don Toliver, Travis Scott - HARDSTONE PSYCHO - 2024-06-14 - 153.99 BPM - 0.77 - 0.70 - 0.53 - https://open.spotify.com/track/4nhcjelU9PidsBzUkazfZs
